In [31]:
from database_util import connect_db, get_uncleaned_html_records, update_cleaned_html

def gethtml(tender_id):
    # 连接数据库
    db = connect_db()
    if db is None:
        print("Failed to connect to the database.")
        return
    # 创建游标
    cursor = db.cursor()
    # 获取当前tender_id下，tender_detail_html表中的cleaned_detail_html字段
    cursor.execute("SELECT tender_id, cleaned_detail_html FROM tender_detail_html WHERE tender_id = %s", (tender_id,))
    result = cursor.fetchone()
    # print(result)
    print("LOAD SQL SUCCESS")
    # 关闭游标和数据库连接
    cursor.close()
    db.close()
    if result is not None:
        return result[1]
        print("SQL gethtml"+tender_id+" SUCCESS AND CLOSED")
    else:
        return None

In [76]:
tt = gethtml(468)
print(tt)

LOAD SQL SUCCESS
  <div><p>广东省工程建设项目招标公告</p><table><tbody><tr><td><p>投资项目代码</p></td><td><p>2202-440783-04-01-935321</p></td></tr><tr><td><p>投资项目名称</p></td><td><p>开平市国际卫浴创新基地产业综合配套工程</p></td></tr><tr><td><p>招标项目名称</p></td><td><p>开平市国际卫浴创新基地产业综合配套工程（一期）全过程工程咨询（勘察、方案及初步设计、监理、全过程造价咨询）</p></td></tr><tr><td><p>标段（包）名称</p></td><td><p>开平市国际卫浴创新基地产业综合配套工程（一期）全过程工程咨询（勘察、方案及初步设计、监理、全过程造价咨询）</p></td><td><p>公告性质</p></td><td><p>正常</p></td></tr><tr><td><p>资格审查方式</p></td><td><p>资格后审</p></td></tr><tr><td><p>招标项目实施</p><p>（交货）地点</p></td><td><p>江门市开平市水口镇国际卫浴创新基地内</p></td></tr><tr><td><p>资金来源</p></td><td><p>自筹资金</p></td><td><p>资金来源构成</p></td><td><p>全部来源自筹资金</p></td></tr><tr><td><p>招标范围及规模</p></td><td><p>招标范围及内容：</p><p>(1)工程勘察招标内容：本项目规划红线图范围内所需的所有勘察、测量工作，确保勘察数据的准确性与有效性，为后续设计提供依据,包括但不限于岩土工程勘察（包括钻孔、取样、现场标贯试验、技术工作、室内试验及其它相关工作）、物探测试检测（地下管线探测、技术工作等）。</p><p>(2)工程设计招标内容：</p><p>①方案设计：根据招标人整体规划、建设要求及实际情况提供本项目方案设计，确保方案设计满足国家现行的有关规定及相关专业工程技术标准和现行设计规范、规程的有关深度和内容要求；</p><p>②初步设计：项目范围内的规划及初步设计，编制工程概算，并提交初步设计成果文件，且通过全部评审。对

In [112]:
from zhipuai import ZhipuAI
from database_util import connect_db,get_all_cleaned_htmls_to_extract,update_last_extracted_time,insert_data_into_tender_detail
import logging
# 配置日志级别，输出位置等
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


def extract_detail_from_html(html):

    client = ZhipuAI(api_key="e6af334544b37a85e83900a9152eb9a0.GzjnBTUhPOhDDJhx")

    tools = [{
        "type": "function",
        "function": {
            "name": "get_tender_detail",
            "description": "从提供的招标公告内容中，提取需要的字段信息。请注意不得编造，无则留空。",
            "parameters": {
            "type": "object",
            "properties": {
                "html": {"description": "提供的招标公告内容", "type": "text"},
                "tender_project_name": {"description": "项目名称：完整的项目名称", "type": "string"},
                "implementation_site": {"description": "施工地点：具体的项目实施地点", "type": "string"},
                "scope_and_range": {"description": "施工规模和范围：工程项目概括及施工类型、监理服务范围", "type": "string"},
                "gross_floor_area": {"description": "总建筑面积：项目的总建筑面积", "type": "string"},
                "total_investment": {"description": "总投资额：项目总投资金额", "type": "string"},
                "duration": {"description": "工期：工程预计完成时间（日历天，月份，或起始和终止日期）", "type": "string"},
                "bid_price": {"description": "监理服务费或最高限价：用于表示项目监理服务费用或监理服务最高报价限制", "type": "string"},
                "bid_price_rate": {"description": "监理服务费率：表示计算最终监理服务费用的系数或比例，以小数表示", "type": "string"},
                "company_qualification_requirement": {"description": "企业资质要求：包括资质类型和资质等级要求", "type": "string"},
                "chief_registration_certificate": {"description": "总监专业要求：注册监理工程师资格证书专业要求", "type": "string"},
                "chief_title_and_education": {"description": "总监职称学历：总监的职称要求和学历要求", "type": "string"},
                "chief_experience": {"description": "总监业绩经验：总监类似项目方面业绩要求，要求候选人至少成功监理过特定数量的相似类型工程", "type": "string"},
                "chief_restrictions": {"description": "总监限制：总监任职限制，如不允许同时监理多个项目", "type": "string"},
                "tenderer": {"description": "招标方：招标方的名称", "type": "string"},
                "bid_opening_time": {"description": "开标时间：开标仪式的具体时间以年月日时分表示", "type": "string"}
            },
            "required": [
                "html",
                "tender_project_name",
                "implementation_site",
                "scope_and_range",
                "gross_floor_area",
                "total_investment",
                "duration",
                "bid_price",
                "bid_price_rate",
                "company_qualification_requirement",
                "chief_registration_certificate",
                "chief_title_and_education",
                "chief_experience",
                "chief_restrictions",
                "tenderer",
                "bid_opening_time"
            ]
            }
        }
        }]

    # 清空对话
    messages = []
    messages.append({"role": "user", "content": "从招标公告内容中，提取招标相关的全部信息。使用get_tender_detail这个tool来匹配对应字段，请注意不得编造，无则留空"})
    messages.append({"role": "system", "content": "当然，请发送所需的招标公告，我会进行提取"})
    messages.append({"role": "user", "content": html})

    
    response = client.chat.completions.create(
        model="glm-4", 
        messages=messages,
        tools=tools,
        tool_choice="auto",
        # tool_choice={"type": "function", "function": {"name": "get_tender_detail"}},
        top_p=0.7,
        temperature=0.95,
        max_tokens=4096,
    )
    
    # logging.info("提取HTML招标详情的响应: {}".format(response))

    return response

In [113]:
test=extract_detail_from_html(tt)
print(test)

2024-03-27 01:01:33,340 - INFO - HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/chat/completions "HTTP/1.1 200 "


model='glm-4' created=1711472493 choices=[CompletionChoice(index=0, finish_reason='stop', message=CompletionMessage(content='根据您提供的招标公告内容，以下是提取的招标相关信息：\n\n- 项目名称：开平市国际卫浴创新基地产业综合配套工程（一期）全过程工程咨询（勘察、方案及初步设计、监理、全过程造价咨询）\n- 施工地点：江门市开平市水口镇国际卫浴创新基地内\n- 施工规模和范围：招标范围及内容：...全过程造价咨询：...。项目建设规模：一期项目用地面积约25亩（16667平方米），地下室面积约10260平方米，总计容建筑面积约41667平方米。包含高度约40m的人才公寓一栋，高度约70m的员工宿舍一栋，以及食堂超市配套。\n- 总建筑面积：总计容建筑面积约41667平方米\n- 总投资额：项目估算总投资72258.65万元，其中一期项目：建安工程费约18000万元\n- 工期：自本项目合同签订之日起，至竣工结算且完成本项目全过程工程服务止（包含所有工程的缺陷责任期）\n- 监理服务费或最高限价：/\n- 监理服务费率：/\n- 企业资质要求：投标人必须具备以下三项的资质：...。\n- 总监专业要求：总监专业要求：...。\n- 总监职称学历：总监职称学历：...。\n- 总监业绩经验：总监业绩经验：...。\n- 总监限制：总监限制：...。\n- 招标方：开平市水口镇人民政府\n- 开标时间：2024年4月16日09 时30分\n\n请注意，有些信息在您提供的内容中没有明确提及，因此标记为“/”或留空。如果需要更详细的信息，请提供完整的招标公告。', role='assistant', tool_calls=None))] request_id='8513523894089077306' id='8513523894089077306' usage=CompletionUsage(prompt_tokens=986, completion_tokens=355, total_tokens=1341)


In [114]:
print(test.choices[0].message.tool_calls)

None


In [102]:
print(test.choices[0].message.content)

根据您提供的招标公告内容，以下是提取的招标相关信息：

- 项目名称：开平市国际卫浴创新基地产业综合配套工程（一期）全过程工程咨询（勘察、方案及初步设计、监理、全过程造价咨询）
- 施工地点：江门市开平市水口镇国际卫浴创新基地内
- 施工规模和范围：招标范围及内容涉及全过程造价咨询。一期项目用地面积约25亩（16667平方米），地下室面积约10260平方米，总计容建筑面积约41667平方米。包含高度约40m的人才公寓一栋，高度约70m的员工宿舍一栋，以及食堂超市配套。
- 总建筑面积：总计容建筑面积约41667平方米
- 总投资额：项目估算总投资72258.65万元，其中一期项目建安工程费约18000万元
- 工期：自本项目合同签订之日起，至竣工结算且完成本项目全过程工程服务止（包含所有工程的缺陷责任期）
- 监理服务费或最高限价：/
- 监理服务费率：/
- 企业资质要求：投标人必须具备以下三项的资质
- 总监专业要求：总监专业要求内容未提供
- 总监职称学历：总监职称学历要求未提供
- 总监业绩经验：总监业绩经验要求未提供
- 总监限制：总监限制条件未提供
- 招标方：开平市水口镇人民政府
- 开标时间：2024年4月16日09时30分

请注意，部分信息未在您提供的公告中明确，因此留空。如果需要更详细的信息，请提供完整的招标公告内容。


In [97]:
# 当test.choices[0].message.tool_calls为空时，将test返回结果中的choices[0].message.content重新赋值给tt并再次运行extract_detail_from_html
if not test.choices[0].message.tool_calls:
    tt = test.choices[0].message.content
    testagain = extract_detail_from_html(tt)
    print(testagain)



2024-03-27 00:25:40,963 - INFO - HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/chat/completions "HTTP/1.1 200 "


model='glm-4' created=1711470340 choices=[CompletionChoice(index=0, finish_reason='stop', message=CompletionMessage(content='以下是从招标公告中提取的相关信息：\n\n- 项目名称：开平市国际卫浴创新基地产业综合配套工程（一期）全过程工程咨询（勘察、方案及初步设计、监理、全过程造价咨询）\n- 施工地点：江门市开平市水口镇国际卫浴创新基地内\n- 施工规模和范围：一期项目用地面积约25亩（16667平方米），地下室面积约10260平方米，总计容建筑面积约41667平方米。包含高度约40m的人才公寓一栋，高度约70m的员工宿舍一栋，以及食堂超市配套。\n- 总建筑面积：总计容建筑面积约41667平方米\n- 总投资额：项目估算总投资72258.65万元，其中一期项目：建安工程费约18000万元\n- 工期：自本项目合同签订之日起，至竣工结算且完成本项目全过程工程服务止（包含所有工程的缺陷责任期）\n- 监理服务费或最高限价：/\n- 监理服务费率：/\n- 企业资质要求：投标人必须具备以下三项的资质：...\n- 总监专业要求：...\n- 总监职称学历：...\n- 总监业绩经验：...\n- 总监限制：...\n- 招标方：开平市水口镇人民政府\n- 开标时间：2024年4月16日09 时30分\n\n请注意，部分信息可能未在招标公告中提供，因此标记为“/”或留空。如果需要更详细的信息，请提供完整的招标公告内容。', role='assistant', tool_calls=None))] request_id='8513530216281032426' id='8513530216281032426' usage=CompletionUsage(prompt_tokens=1042, completion_tokens=317, total_tokens=1359)


In [98]:
print(testagain.choices[0].message.tool_calls)

None
